# DA Course | Lesson-1

In [ ]:
who = !whoami
print(who[0])

In [ ]:
import os

daily_engagement_filename = os.path.join('/', 'Users', who[0], 'Documents', 'data', 'daily_engagement.csv')
submissions_filename = os.path.join('/', 'Users', who[0], 'Documents', 'data', 'project_submissions.csv')
enrollments_filename = os.path.join('/', 'Users', who[0], 'Documents', 'data', 'enrollments.csv')
print(daily_engagement_filename)
print(submissions_filename)
print(enrollments_filename)

In [ ]:
# function to read csv file
import unicodecsv

def read_csv_file(file_path):
    with open(file_path, 'rb') as f:
        reader = unicodecsv.DictReader(f)
        return list(reader)

daily_engagement = read_csv_file(daily_engagement_filename)
submissions = read_csv_file(submissions_filename)
enrollments = read_csv_file(enrollments_filename)

In [ ]:
# import datetime
# now = datetime.datetime.now()
daily_engagement[0]
submissions[0]
enrollments[0]

In [ ]:
!pwd
!whoami

# Fixing data types

In [ ]:
from datetime import datetime as dt

# Takes a date as a string, and returns a Python datetime object. 
# If there is no date given, returns None
def parse_date(date):
    if date == '':
        return None
    else:
        return dt.strptime(date, '%Y-%m-%d')
    
# Takes a string which is either an empty string or represents an integer,
# and returns an int or None.
def parse_maybe_int(i):
    if i == '':
        return None
    else:
        return int(i)

# Clean up the data types in the enrollments table
for enrollment in enrollments:
    enrollment['cancel_date'] = parse_date(enrollment['cancel_date'])
    enrollment['days_to_cancel'] = parse_maybe_int(enrollment['days_to_cancel'])
    enrollment['is_canceled'] = enrollment['is_canceled'] == 'True'
    enrollment['is_udacity'] = enrollment['is_udacity'] == 'True'
    enrollment['join_date'] = parse_date(enrollment['join_date'])
    
enrollments[0]

In [ ]:
# Clean up the data types in the engagement table
for engagement_record in daily_engagement:
    engagement_record['lessons_completed'] = int(float(engagement_record['lessons_completed']))
    engagement_record['num_courses_visited'] = int(float(engagement_record['num_courses_visited']))
    engagement_record['projects_completed'] = int(float(engagement_record['projects_completed']))
    engagement_record['total_minutes_visited'] = float(engagement_record['total_minutes_visited'])
    engagement_record['utc_date'] = parse_date(engagement_record['utc_date'])
    engagement_record['account_key'] = engagement_record['acct']
    del[engagement_record['acct']]
    
daily_engagement[0]

In [ ]:
# Clean up the data types in the submissions table
for submission in submissions:
    submission['completion_date'] = parse_date(submission['completion_date'])
    submission['creation_date'] = parse_date(submission['creation_date'])

submissions[0]

In [58]:
def get_unique_students(data):
    unique_students = set()
    for data_point in data:
        unique_students.add(data_point['account_key'])
    return unique_students

In [60]:
len(enrollments)

1640

In [62]:
unique_enrolled_students = set()
for enrollment in enrollments:
    unique_enrolled_students.add(enrollment['account_key'])
len(unique_enrolled_students)

1302

In [63]:
unique_project_submissions = get_unique_students(submissions)
unique_daily_engagements = get_unique_students(daily_engagement)

In [65]:
len(unique_project_submissions)
len(unique_daily_engagements)

743

## Investigating the data

In [ ]:
#####################################
#                 2                 #
#####################################

## Find the total number of rows and the number of unique students (account keys)
## in each table.



## Problems in data

In [85]:
#####################################
#                 3                 #
#####################################

## Rename the "acct" column in the daily_engagement table to "account_key".
# for engagement in daily_engagement:
#     engagement['account_key'] = engagement['acct']
#     del[engagement['acct']]

daily_engagement[0]

{'utc_date': datetime.datetime(2015, 1, 9, 0, 0),
 'num_courses_visited': 1,
 'total_minutes_visited': 11.6793745,
 'lessons_completed': 0,
 'projects_completed': 0,
 'account_key': '0'}

## Missing Engagement Records

In [91]:
#####################################
#                 4                 #
#####################################

## Find any one student enrollments where the student is missing from the daily engagement table.
## Output that enrollment.
# for engag in daily_engagement:
#     print(engag['account_key'])

count = 0
for enrol in enrollments:
    if enrol['account_key'] not in unique_daily_engagements:
        count += 1
        print(enrol)
        break

print(count)

#
#print(next(iter(enrollments))

{'account_key': '1219', 'status': 'canceled', 'join_date': datetime.datetime(2014, 11, 12, 0, 0), 'cancel_date': datetime.datetime(2014, 11, 12, 0, 0), 'days_to_cancel': 0, 'is_udacity': False, 'is_canceled': True}
1


## Checking for More Problem Records

In [95]:
#####################################
#                 5                 #
#####################################

## Find the number of surprising data points (enrollments missing from
## the engagement table) that remain, if any.
count = 0
for enrol in enrollments:
    if enrol['account_key'] not in unique_daily_engagements and enrol['join_date'] != enrol['cancel_date']:
        count += 1
        print(enrol)
        break

print(count)

{'account_key': '1304', 'status': 'canceled', 'join_date': datetime.datetime(2015, 1, 10, 0, 0), 'cancel_date': datetime.datetime(2015, 3, 10, 0, 0), 'days_to_cancel': 59, 'is_udacity': True, 'is_canceled': True}
1


## Tracking Down the Remaining Problems

In [97]:
# Create a set of the account keys for all Udacity test accounts
udacity_test_accounts = set()
for enrol in enrollments:
    if enrol['is_udacity'] == True:
        udacity_test_accounts.add(enrol['account_key'])
        
len(udacity_test_accounts)
    

6

In [101]:
# Given some data with an account_key field, removes any records corresponding to Udacity test accounts
def remove_udacity_accounts(data):
    non_udacity_data = []
    for data_point in data:
        if data_point['account_key'] not in udacity_test_accounts:
            non_udacity_data.append(data_point)
    return non_udacity_data

In [102]:
# Remove Udacity test accounts from all three tables
non_udacity_enrollments = remove_udacity_accounts(enrollments)
non_udacity_engagement = remove_udacity_accounts(daily_engagement)
non_udacity_submissions = remove_udacity_accounts(submissions)

print(len(non_udacity_enrollments))
print(len(non_udacity_engagement))
print(len(non_udacity_submissions))

1622
135656
3634


## Refining the Question

In [110]:
#####################################
#                 6                 #
#####################################

## Create a dictionary named paid_students containing all students who either
## haven't canceled yet or who remained enrolled for more than 7 days. The keys
## should be account keys, and the values should be the date the student enrolled.

paid_students = {}

for enrol in non_udacity_enrollments:
    if not enrol['is_canceled'] or enrol['days_to_cancel'] > 7:
        accnt_key = enrol['account_key']
        enrol_date = enrol['join_date']
        
        if accnt_key not in paid_students or enrol_date > paid_students[accnt_key]:   
            paid_students[accnt_key] = enrol_date
        
print(len(paid_students))

995


## Getting Data from First Week

In [111]:
# Takes a student's join date and the date of a specific engagement record,
# and returns True if that engagement record happened within one week
# of the student joining.
def within_one_week(join_date, engagement_date):
    time_diff = engagement_date - join_date
    return time_diff.days < 7

In [131]:
# create a function to remove trial students
def remove_free_trial_cancels(data):
    new_data = []
    for data_point in data:
        if data_point['account_key'] in paid_students:
            new_data.append(data_point)
    return new_data

In [134]:
paid_enrollments = remove_free_trial_cancels(non_udacity_enrollments)
paid_engagement = remove_free_trial_cancels(non_udacity_engagement)
paid_submissions = remove_free_trial_cancels(non_udacity_submissions)

print(len(paid_enrollments))
print(len(paid_engagement))
print(len(paid_submissions))


1293
134549
3618


In [129]:
#####################################
#                 7                 #
#####################################

## Create a list of rows from the engagement table including only rows where
## the student is one of the paid students you just found, and the date is within
## one week of the student's join date.

paid_engagement_in_first_week = []

for engagmnt in non_udacity_engagement:
    # paid = paid_students[engagmnt['account_key']]
    if engagmnt['account_key'] in paid_students \
    and within_one_week(paid_students[engagmnt['account_key']], engagmnt['utc_date']):
            paid_engagement_in_first_week.append(engagmnt)
        
print(len(paid_engagement_in_first_week))


21508


## Exploring Student Engagement

In [182]:
for paid in paid_enrollments:
    print(paid)
    break
    

{'account_key': '429', 'status': 'canceled', 'join_date': datetime.datetime(2014, 11, 10, 0, 0), 'cancel_date': datetime.datetime(2015, 3, 10, 0, 0), 'days_to_cancel': 120, 'is_udacity': False, 'is_canceled': True}


In [177]:
# # Create a dict of paid udacity students who stayped past the trial period
# paid_engagement_post_first_week = {}
# engagement_by_account = defaultdict(list)
# for engag in non_udacity_engagement:
# #     print(engag)
# #     break
#     if engag == [v for k, v in paid_engagement_in_first_week.items()]
#     # if paid_engagement_in_first_week[''] 
#     if engag not in paid_engagement_in_first_week:
#         engagement_by_account[engag['account_key']].append({k: v for k, v in engag.items() if k != 'account_key'})
        
# len(engagement_by_account)



{'utc_date': datetime.datetime(2015, 1, 9, 0, 0), 'num_courses_visited': 1, 'total_minutes_visited': 11.6793745, 'lessons_completed': 0, 'projects_completed': 0, 'account_key': '0'}


0

In [185]:
from collections import defaultdict

# Create a dictionary of engagement grouped by student.
# The keys are account keys, and the values are lists of engagement records.
engagement_by_account = defaultdict(list)
for engagement_record in paid_engagement:
    account_key = engagement_record['account_key']
    engagement_by_account[account_key].append(engagement_record)
    
len(engagement_by_account)
    

995

In [173]:
count = 0
for engag in paid_engagement_in_first_week:
    if int(engag['account_key']) == 0:
        count += 1
    
print(count)


>>>>>
>>>>>
>>>>>
>>>>>
>>>>>
>>>>>
>>>>>
7


In [186]:
for engag in engagement_by_account.items():
    print(engag)
    break
    
# paid_list = [v for k, v in paid_enrollments]
# print(paid_list)


('0', [{'utc_date': datetime.datetime(2015, 1, 9, 0, 0), 'num_courses_visited': 1, 'total_minutes_visited': 11.6793745, 'lessons_completed': 0, 'projects_completed': 0, 'account_key': '0'}, {'utc_date': datetime.datetime(2015, 1, 10, 0, 0), 'num_courses_visited': 2, 'total_minutes_visited': 37.2848873333, 'lessons_completed': 0, 'projects_completed': 0, 'account_key': '0'}, {'utc_date': datetime.datetime(2015, 1, 11, 0, 0), 'num_courses_visited': 2, 'total_minutes_visited': 53.6337463333, 'lessons_completed': 0, 'projects_completed': 0, 'account_key': '0'}, {'utc_date': datetime.datetime(2015, 1, 12, 0, 0), 'num_courses_visited': 1, 'total_minutes_visited': 33.4892696667, 'lessons_completed': 0, 'projects_completed': 0, 'account_key': '0'}, {'utc_date': datetime.datetime(2015, 1, 13, 0, 0), 'num_courses_visited': 1, 'total_minutes_visited': 64.7796776667, 'lessons_completed': 0, 'projects_completed': 0, 'account_key': '0'}, {'utc_date': datetime.datetime(2015, 1, 14, 0, 0), 'num_course

In [187]:
# Create a dictionary with the total minutes each student spent in the classroom during the first week.
# The keys are account keys, and the values are numbers (total minutes)
total_minutes_by_account = {}
for account_key, engagement_for_student in engagement_by_account.items():
    total_minutes = 0
    for engagement_record in engagement_for_student:
        total_minutes += engagement_record['total_minutes_visited']
    total_minutes_by_account[account_key] = total_minutes
    
len(total_minutes_by_account)

995

In [194]:
import numpy as np

# Summarize the data about minutes spent in the classroom
total_minutes = total_minutes_by_account.values()
print('Mean:', np.mean(total_minutes))
print('Standard deviation:', np.std(total_minutes))
print('Minimum:', np.min(total_minutes))
print('Maximum:', np.max(total_minutes))

TypeError: int() argument must be a string, a bytes-like object or a number, not 'dict_values'

In [193]:
print(total_minutes)

dict_values([7605.371410665008, 4296.2444758356305, 2977.4300446686007, 4422.018918834502, 3722.648850166561, 4908.933278333178, 4106.3419510003705, 2196.9374261675703, 4620.633675333883, 6931.9974760003615, 6786.522830667709, 601.0891874993999, 10586.66412016477, 1977.09014266663, 2171.3060355006, 9278.582126833042, 1446.9394143341997, 2620.4063588342397, 1966.8141853335999, 2011.20928533247, 2324.0164696666593, 2212.98379266677, 2659.4834441669996, 10414.645088664402, 8112.847323668319, 4840.034289498201, 2963.45045933326, 5136.50362650046, 7448.364715667938, 2272.69812616771, 3839.7972376664993, 3433.4410844989993, 3767.4314579999614, 7899.296711499474, 3640.8049636655896, 6720.716051333241, 2481.7975725013, 13244.547495498631, 13585.694861667058, 2317.4549943339703, 1580.7553903333298, 2909.2735703322296, 2713.7918413321595, 1665.9478503340597, 4270.82292883184, 8970.50406050229, 5946.2192736669995, 1672.7847229999998, 2388.955935667, 852.7138903325999, 6793.815464836333, 2280.1874